In [10]:
import sagemaker
from sagemaker.workflow.notebook_job_step import NotebookJobStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker import session

sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
subfolder_name = "notebook-step-artifacts-pipelines/"
image_uri ="908471171231.dkr.ecr.us-east-1.amazonaws.com/sagemaker-images:sm-distribution-custom-img-v2"
kernel_name = "python3"
role = sagemaker.get_execution_role()
notebook_artifacts = f"s3://{default_bucket}/{subfolder_name}"
notebook_artifacts

's3://sagemaker-us-east-1-908471171231/notebook-step-artifacts-pipelines/'

In [16]:
pipeline_name = "nb-job-step-pipelines-demo3"
display_name = "MyNotebookSteps"
preprocess_notebook = "test_.ipynb"
preprocess_job_name = "test"
preprocess_description = "test to save file locally"
preprocess_step_name ="test"

# notebook job parameters
nb_job_params = {"default_s3_bucket": notebook_artifacts}

preprocess_nb_step = NotebookJobStep(
    name=preprocess_step_name,
    description=preprocess_description,
    notebook_job_name=preprocess_job_name,
    image_uri=image_uri,
    kernel_name=kernel_name,
    display_name=display_name,
    role=role,
    input_notebook=preprocess_notebook,
    instance_type="ml.m5.large",
    parameters=nb_job_params,
    additional_dependencies=['requirements.txt'],
)

In [17]:
# create pipeline
pipeline = Pipeline(
    name=pipeline_name,
    sagemaker_session=sagemaker_session,
    steps=[preprocess_nb_step],
)

In [18]:
pipeline.upsert(session.get_execution_role())
execution = pipeline.start(parameters={})
execution.wait(delay=50, max_attempts=60)
execution_steps = execution.list_steps()
print(execution_steps)

[{'StepName': 'test', 'StartTime': datetime.datetime(2024, 5, 21, 15, 20, 7, 329000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 5, 21, 15, 23, 42, 498000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:908471171231:training-job/test-svwkei9eamir-9w8HuvoRNI'}}, 'AttemptCount': 1}]


In [21]:
# download the output notebook

from sagemaker.s3_utils import s3_path_join
from sagemaker.utils import _tmpdir
from sagemaker.s3 import S3Downloader
import tarfile
import os


# get job details
def _get_training_job_details(notebook_job_step):
    training_job_arn = notebook_job_step["Metadata"]["TrainingJob"]["Arn"]

    return sagemaker_session.sagemaker_client.describe_training_job(
        TrainingJobName=training_job_arn.split("/")[1]
    )


def _download_notebook(output_s3_uri, output_notebook_name, kms_key=None):
    download_folder = "./outputs"

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    
    S3Downloader.download(
        output_s3_uri,
        download_folder,
        sagemaker_session=sagemaker_session,
        #kms_key=kms_key,
    )

    with tarfile.open(os.path.join(download_folder, "output.tar.gz"), "r:gz") as tar:
        tar.extractall('./')
        print(f"Downloaded to {download_folder}/{output_notebook_name}")


# download the output notebook job
job_description = _get_training_job_details(execution_steps[0])

output_s3_uri = s3_path_join(
    job_description["OutputDataConfig"]["S3OutputPath"],
    job_description["TrainingJobName"],
    "output",
    "output.tar.gz",
)
output_notebook_name = job_description["Environment"]["SM_OUTPUT_NOTEBOOK_NAME"]

print(f"  - Output S3 Location: {output_s3_uri}")
print(f"  - Output Notebook Name: {output_notebook_name}")

_download_notebook(output_s3_uri, output_notebook_name)

  - Output S3 Location: s3://sagemaker-us-east-1-908471171231/nb-job-step-pipelines-demo3/svwkei9eamir/test/test-svwkei9eamir-9w8HuvoRNI/output/output.tar.gz
  - Output Notebook Name: test.ipynb
Downloaded to ./outputs/test.ipynb


In [24]:
from sagemaker.workflow.triggers import PipelineSchedule

# schedules a pipeline run every 5 minutes
my_rate_schedule = PipelineSchedule(
    name="savefile-every5min", 
    rate=(5, "minutes")
)

pipeline.put_triggers(triggers=[my_rate_schedule], role_arn=role)

INFO:sagemaker.workflow.pipeline:Creating/Updating EventBridge Schedule for pipeline: nb-job-step-pipelines-demo3.


ClientError: An error occurred (AccessDeniedException) when calling the UpdateSchedule operation: User: arn:aws:sts::908471171231:assumed-role/dre-poc-nonprod-sagemaker-execution-role/SageMaker is not authorized to perform: scheduler:UpdateSchedule on resource: arn:aws:scheduler:us-east-1:908471171231:schedule/default/savefile-every5min because no identity-based policy allows the scheduler:UpdateSchedule action

In [ ]:
datascience team  = Analysis_script.py
datascience lead = job.py(which calls above scripty)

jenkins pipeline should schedule job.py script without having us to run from sagemaker


puzzle: how to trigger job.py outside sagemaker
